# Importa bibliotecas

In [1]:
import pandas as pd
import glob
import numpy as np
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Masking
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from keras.layers import Dropout
from sklearn.preprocessing import OneHotEncoder
from keras.callbacks import EarlyStopping
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

2025-07-08 19:51:28.011306: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752015088.031485    6745 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752015088.038239    6745 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752015088.052473    6745 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752015088.052515    6745 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752015088.052535    6745 computation_placer.cc:177] computation placer alr

## Definições

In [2]:
import os
import random
import tensorflow as tf

seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# Preparando dados

In [3]:
csv_files = glob.glob('/mnt/d/dados_surdos/CSVs/dados_pessoa2_*.csv')

dfs = []
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)
df

,landmark_0_x,landmark_0_y,landmark_0_z,landmark_1_x,landmark_1_y,landmark_1_z,landmark_2_x,landmark_2_y,landmark_2_z,landmark_3_x,...,landmark_18_z,landmark_19_x,landmark_19_y,landmark_19_z,landmark_20_x,landmark_20_y,landmark_20_z,repetition,frame,word
0,-0.015607,-0.012011,0.085424,0.002198,0.002056,0.062723,0.006184,0.013907,0.043209,0.012581,...,0.010308,-0.014324,0.022577,0.024143,-0.012121,0.028077,0.045234,0,0,amanda
1,-0.026881,-0.016575,0.088360,-0.004162,0.000434,0.064802,-0.002506,0.012053,0.047482,0.012059,...,0.006520,-0.015839,0.031193,0.016019,-0.011932,0.045492,0.040148,0,1,amanda
2,-0.035663,-0.030682,0.055793,-0.007602,-0.020255,0.040326,0.010573,-0.015109,0.029176,0.034369,...,0.005534,-0.019610,0.026664,0.005605,-0.010181,0.035500,0.006776,0,2,amanda
3,-0.019877,-0.012865,0.072611,-0.000591,-0.004493,0.054639,0.013915,0.000713,0.042367,0.027604,...,-0.000779,-0.025155,0.012071,-0.001725,-0.019246,0.009204,0.004844,0,3,amanda
4,-0.017428,0.004787,0.097933,0.009870,0.017381,0.064847,0.028842,0.017987,0.040486,0.040202,...,0.009571,-0.020957,0.031525,0.010485,-0.019979,0.045894,0.023430,0,4,amanda
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,-0.006988,0.059106,0.065497,0.018958,0.042008,0.045603,0.036582,0.031468,0.027750,0.041668,...,0.002797,-0.024219,-0.000852,0.008091,-0.022274,0.007363,0.013810,9,147,zimbabwe
32557,-0.009849,0.055787,0.067987,0.014397,0.041232,0.047113,0.037282,0.032145,0.029211,0.048260,...,-0.007995,-0.022478,0.014994,-0.003822,-0.018179,0.023010,0.001642,9,148,zimbabwe
32558,-0.010559,0.020268,0.080483,0.017107,0.022272,0.055852,0.036796,0.022708,0.033825,0.051180,...,0.001762,-0.009830,0.034790,0.009860,-0.008334,0.042102,0.022468,9,149,zimbabwe
32559,-0.009250,-0.018638,0.095681,0.017051,-0.004999,0.066634,0.035379,0.004927,0.049955,0.049636,...,0.011627,-0.027174,0.028498,0.016777,-0.027771,0.039360,0.024744,9,150,zimbabwe


In [4]:
# Separa features e label
grouped = df.groupby(['word', 'repetition'])
X_raw = []
y_raw = []

In [5]:
# Normaliza features com valores entre 0 e 1
scaler = MinMaxScaler()
landmark_cols = list(df.columns[:63])
df[landmark_cols] = scaler.fit_transform(df[landmark_cols])

In [6]:
# prepara lista com frames agrupadas por video
for (word, rep), group in grouped:
    sequence = group[landmark_cols].values
    X_raw.append(sequence)
    y_raw.append(word)

In [7]:
# Separa em treino, teste e validação de forma estratificada

X_temp, X_test, y_temp, y_test = train_test_split(
    X_raw, y_raw, test_size=0.3, stratify=y_raw)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.285, stratify=y_temp)

In [8]:
print(f"Treino - {(len(X_train)/len(X_raw))*100}%")
print(f"Teste - {(len(X_test)/len(X_raw))*100}%")
print(f"Validacao - {(len(X_val)/len(X_raw))*100}%")

Treino - 50.0%
Teste - 30.0%
Validacao - 20.0%


X_raw é uma lista de sequências de frames, onde:

- cada item da lista representa um vídeo.

- cada vídeo é representado como um array 2D de shape (T, 63), onde:

    - T = número de frames (time steps) do vídeo (varia de vídeo para vídeo)

    - 63 = número de features por frame (21 pontos da mão × 3 coordenadas)

In [9]:
# Padding
max_len = max(len(seq) for seq in X_train) # define tamanho maximo das sequencias

X_train = pad_sequences(X_train, maxlen=max_len, padding='post', dtype='float32')
X_val = pad_sequences(X_val,   maxlen=max_len, padding='post', dtype='float32')
X_test = pad_sequences(X_test,  maxlen=max_len, padding='post', dtype='float32')

X_train.shape,X_val.shape,X_test.shape

((130, 247, 63), (52, 247, 63), (78, 247, 63))

In [10]:
X_train.shape

(130, 247, 63)

In [11]:
# Encode das labels - OneHotEncoder
label_encoder = OneHotEncoder(sparse_output=False)
y_train = label_encoder.fit_transform(np.array(y_train).reshape(-1, 1))
y_val = label_encoder.transform(np.array(y_val).reshape(-1, 1))
y_test = label_encoder.transform(np.array(y_test).reshape(-1, 1))

y_train.shape,y_test.shape,y_val.shape

((130, 26), (78, 26), (52, 26))

# Criando o modelo

In [12]:
num_classes = 26 # palavras/labels

In [13]:
mask = Masking(mask_value=0.0, input_shape=(max_len, 63))
SRNN = SimpleRNN(128)
dense = Dense(num_classes, activation='softmax')

model = Sequential()
model.add(mask)
model.add(SRNN)
model.add(Dropout(0.3))
model.add(dense)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

/home/victor/.local/lib/python3.10/site-packages/keras/src/layers/core/masking.py:48: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
2025-07-08 19:51:33.289082: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, 247, 63)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 128)            │        24,576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 26)             │         3,354 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,930 (109.10 KB)

 Trainable params: 27,930 (109.10 KB)

 Non-trainable params: 0 (0.00 B)

# Treinamento

In [14]:
epochs = 10000

history = model.fit(
    X_train, y_train,
    batch_size=8,
    epochs=epochs,
    verbose=1,
    validation_data=(X_val, y_val),
    shuffle=False
)

Epoch 1/10000


2025-07-08 19:51:33.461864: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}


17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - accuracy: 0.0243 - loss: 3.5250 - val_accuracy: 0.1154 - val_loss: 3.0832
Epoch 2/10000
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.1345 - loss: 3.0426 - val_accuracy: 0.0962 - val_loss: 2.7120
Epoch 3/10000
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.1865 - loss: 2.7102 - val_accuracy: 0.2115 - val_loss: 2.6083
Epoch 4/10000
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.1673 - loss: 2.6505 - val_accuracy: 0.1731 - val_loss: 2.5010
Epoch 5/10000
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.1930 - loss: 2.4920 - val_accuracy: 0.2308 - val_loss: 2.3273
Epoch 6/10000
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.2097 - loss: 2.4017 - val_accuracy: 0.1346 - val_loss: 2.3994
Epoch 7/10000
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.2306 - loss: 2.3133 - val_accuracy: 0.1923 - val_loss: 2.3141
Epoch 8/10000
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.2334 - loss: 2.3274 - val_accuracy: 

KeyboardInterrupt: 

# Avaliação do modelo

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\naccuracy: {test_accuracy:.4f}")

In [ ]:
# historico de treinamento
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()